In [99]:
import pandas as pd

In [100]:
df = pd.read_csv('combined_scrape_2.csv')

In [101]:
df.shape

(3543710, 4)

In [102]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [103]:
df_target_train = pd.read_csv('y_yangorno_train.csv')

In [104]:
df_target_train.shape

(2449, 2)

In [105]:
df_target_test = pd.read_csv('y_yangorno_test.csv')

In [106]:
df_target_test.shape

(817, 2)

In [107]:
df_target = pd.concat([df_target_test,df_target_train])

In [108]:
df_target.shape

(3266, 2)

In [109]:
users = list(df_target['Unnamed: 0'])

In [110]:
def clean_df(row):
    if row['Author'] in users:
        return '1'
    else:
        return '0'


In [111]:
df['keep'] = df.apply(clean_df,axis=1)

In [112]:
df = df[df['keep']=='1']

In [113]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [114]:
list_of_subs = ['neoliberal','movies','atheism','MMA','technology','Libertarian','teenagers','SquaredCircle',
    'television','science','conspiracy','Futurology','fantasyfootball','samharris','hockey','JoeRogan','exmormon',
    'teslamotors','marvelstudios','hiphopheads','survivor','rupaulsdragrace','Economics','Music','magicTCG',
    'vegan','trees','anime','apple','environment','space','keto','askgaybros','StarWars','Android','TwoXChromosomes',
    'Christianity','formula1','collapse','investing','aznidentity','books','JordanPeterson','gratefuldead','PS4',
    'BasicIncome','barstoolsports','startrek','cars','Tinder','liberalgunowners','DunderMifflin','Cricket',
    'NASCAR','TheLastAirbender','Documentaries','HomeImprovement','sports','Fitness','Fuckthealtright',
    'ENLIGHTENEDCENTRISM','LateStageCapitalism','ABoringDystopia','baseball','redsox','NewYorkMets','NYYankees',
    'nba','CollegeBasketball','lakers','bostonceltics','Conservative','tuesday','CryptoCurrency','Bitcoin',
    'sysadmin','cscareerquestions','nfl','CFB','CHIBears','freefolk','gameofthrones','asoiaf','stupidpol',
    'buildapcsales','Amd','soccer','MLS','Games','gaming']


In [115]:
features = {}
for user in users:
    feature_dict = {}
    for sub in sum_series[user].index:
        if sub in list_of_subs:
            feature_dict[sub]=sum_series[user][sub]
    if len(feature_dict)!=0:
        features[user]=feature_dict


In [116]:
df_feature = pd.DataFrame(features).T

In [117]:
df_feature.head()

,ABoringDystopia,Amd,Android,BasicIncome,Bitcoin,CFB,CHIBears,Christianity,CollegeBasketball,Conservative,...,stupidpol,survivor,sysadmin,technology,teenagers,television,teslamotors,trees,tuesday,vegan
ataraxia77,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
EVEOpalDragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN
Ac9ts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gawbledeeguk,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
CatJongUn,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,5.0,6.0,NaN,1.0,NaN,1.0


In [118]:
df_feature.shape

(2828, 90)

In [119]:
df_feature.fillna(0,inplace=True)

In [120]:
#add code to combine like minded subs
df_feature['anti_alt_right_comb']=df_feature['ENLIGHTENEDCENTRISM']+df_feature['Fuckthealtright']
df_feature['bad_capitalism_comb']=df_feature['LateStageCapitalism']+df_feature['ABoringDystopia']
df_feature['baseball_comb']=df_feature['baseball']+df_feature['redsox']+df_feature['NYYankees']+df_feature['NewYorkMets']
df_feature['basketball_comb']=df_feature['nba']+df_feature['CollegeBasketball']+df_feature['lakers']+df_feature['bostonceltics']
df_feature['conservative_comb']=df_feature['Conservative']+df_feature['tuesday']
df_feature['crypto_comb']=df_feature['CryptoCurrency']+df_feature['Bitcoin']
df_feature['CS_comb']=df_feature['sysadmin']+df_feature['cscareerquestions']
df_feature['football_comb']=df_feature['nfl']+df_feature['CFB']+df_feature['CHIBears']
df_feature['got_comb']=df_feature['freefolk']+df_feature['gameofthrones']+df_feature['asoiaf']
df_feature['pc_comb']=df_feature['buildapcsales']+df_feature['Amd']
df_feature['soccer_comb']=df_feature['soccer']+df_feature['MLS']
df_feature['gaming_comb']=df_feature['gaming']+df_feature['Games']

In [121]:
# ,'Games','gaming','leagueoflegends','NintendoSwitch','hearthstone','Overwatch','2007scape','smashbros','fo76',
#     'apexlegends','pathofexile','FortNiteBR','DestinyTheGame','GlobalOffensive','Destiny','PUBATTLEGROUNDS',
#     'xboxone','DotA2','pcgaming','heroesofthestorm','NoMansSkyTheGame','reddeadredemption']

In [122]:
#here's where i left off, next up is drop combined columns, then drop empyt rows then and div

In [123]:
#remove above single subs
df_feature.drop(['ENLIGHTENEDCENTRISM','Fuckthealtright','LateStageCapitalism','ABoringDystopia','NewYorkMets','redsox',
                 'baseball','NYYankees','nba','CollegeBasketball','lakers','bostonceltics','Conservative','tuesday',
                 'CryptoCurrency','Bitcoin','sysadmin','cscareerquestions','nfl','CFB','CHIBears',
                 'freefolk','gameofthrones','asoiaf','soccer','MLS','buildapcsales','Amd','Games','gaming'],axis=1,inplace=True)

In [124]:
df_feature = df_feature.div(df_feature.sum(axis=1), axis=0)

In [125]:
df_feature.head(20)

,Android,BasicIncome,Christianity,Cricket,Documentaries,DunderMifflin,Economics,Fitness,Futurology,HomeImprovement,...,baseball_comb,basketball_comb,conservative_comb,crypto_comb,CS_comb,football_comb,got_comb,pc_comb,soccer_comb,gaming_comb
ataraxia77,0.000000,0.038462,0.076923,0.0,0.000000,0.000000,0.000000,0.115385,0.038462,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
EVEOpalDragon,0.000000,0.000000,0.000000,0.0,0.196970,0.015152,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.015152,0.000000,0.000000,0.090909
Ac9ts,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000
gawbledeeguk,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000
CatJongUn,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.011494,0.000000,0.011494,0.0,...,0.0,0.022989,0.000000,0.011494,0.0,0.057471,0.091954,0.000000,0.000000,0.206897
lifelovingliberal,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
wellshire,0.000000,0.000000,0.000000,0.0,0.005391,0.018868,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.002695,0.000000,0.0,0.000000,0.393531,0.000000,0.000000,0.024259
Kroz83,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.013699,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.068493,0.000000,0.000000,0.068493
repete2024,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
SkepticalJohn,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [126]:
df_target_train = df_target_train.set_index('Unnamed: 0')

In [127]:
df_target_test = df_target_test.set_index('Unnamed: 0')

In [128]:
df_feature.shape

(2828, 72)

In [129]:
df_train = pd.concat([df_feature,df_target_train],axis=1,join='inner')

In [130]:
df_test = pd.concat([df_feature,df_target_test],axis=1,join='inner')

In [131]:
df_train.shape

(2103, 73)

In [132]:
df_test.shape

(725, 73)

In [133]:
from sklearn.linear_model import LogisticRegression

In [134]:
lr = LogisticRegression()

In [135]:
X_train = df_train.drop('target',axis=1)

In [136]:
y_train = df_train['target']

In [137]:
X_test = df_test.drop('target',axis=1)

In [138]:
y_test = df_test['target']

In [139]:
lr.fit(X_train,y_train)

/Users/yichenh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [140]:
lr.score(X_train,y_train)

0.7755587256300523

In [141]:
y_train.value_counts(normalize=True)

0    0.749406
1    0.250594
Name: target, dtype: float64

In [142]:
lr.score(X_test,y_test)

0.7848275862068965

In [143]:
y_test.value_counts(normalize=True)

0    0.755862
1    0.244138
Name: target, dtype: float64

In [144]:
pd.Series(lr.predict(X_test)).value_counts(normalize=True)

0    0.935172
1    0.064828
dtype: float64

In [145]:
results = pd.DataFrame(lr.coef_.T,X_test.columns)

In [146]:
results

,0
Android,0.707239
BasicIncome,3.464635
Christianity,-0.681448
Cricket,0.690768
Documentaries,-0.129593
DunderMifflin,-0.025790
Economics,-0.299324
Fitness,-0.962670
Futurology,1.951988
HomeImprovement,-0.714654


In [147]:
results.describe()

,0
count,72.000000
mean,-0.015004
std,1.077917
min,-2.354886
25%,-0.726366
50%,-0.140629
75%,0.524657
max,3.464635


In [148]:
results[abs(results[0])<.4]

,0
Documentaries,-0.129593
DunderMifflin,-0.025790
Economics,-0.299324
SquaredCircle,-0.070606
TwoXChromosomes,-0.151666
apple,-0.010308
barstoolsports,0.207174
collapse,0.124822
conspiracy,-0.236691
environment,-0.390175


In [ ]:
#let's run through this notebook again with these columns dropped